In [1]:
from __future__ import print_function
import torch as t

In [12]:
import numpy as np

In [2]:
# 在创建tensor的时候指定requires_grad
a = t.randn(3, 4, requires_grad=True)
# or
a = t.randn(3, 4).requires_grad_()
# or
a = t.randn(3, 4)
a.requires_grad=True
a

tensor([[ 0.4048,  1.8911,  0.2317, -0.9364],
        [ 0.0653,  1.6988, -1.3301,  0.6945],
        [ 1.1976,  2.1658, -0.3343,  1.9798]], requires_grad=True)

In [3]:
b = t.zeros(3, 4).requires_grad_()
b

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], requires_grad=True)

In [4]:
c = a.add(b)  # or c = a + b
c

tensor([[ 0.4048,  1.8911,  0.2317, -0.9364],
        [ 0.0653,  1.6988, -1.3301,  0.6945],
        [ 1.1976,  2.1658, -0.3343,  1.9798]], grad_fn=<AddBackward0>)

In [5]:
d = c.sum()
d.backward()

In [6]:
d # d还是一个requires_grad=True的tensor,对它的操作需要慎重
d.requires_grad
d

tensor(7.7287, grad_fn=<SumBackward0>)

In [7]:
a.grad

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [8]:
# 此处虽然没有指定c需要求导，但是c依赖于a，而a需要求导，
# 因此c的requires_grad属性会自动设置为true
a.requires_grad, b.requires_grad, c.requires_grad

(True, True, True)

In [9]:
a.is_leaf, b.is_leaf, c.is_leaf

(True, True, False)

In [10]:
# c.grad是None， 因c不是叶子节点，它的梯度是用来计算a的梯度
# 虽然c.requires_grad = True， 但是其梯度计算之后即被释放
c.grad is None

<ipython-input-10-b66ad2f15edd>:3: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  c.grad is None


True

计算下面这个函数的导函数：
$$
y = x^2\bullet e^x
$$
它的导函数是：
$$
{dy \over dx} = 2x\bullet e^x + x^2 \bullet e^x
$$
来看看autograd的计算结果与手动求导计算结果的误差。

In [15]:
def f(x):
    # return np.power(x, 2) * np.exp(x)
    return x ** 2 * t.exp(x)

def grad_f(x):
    # return 2 * x * np.exp(x) + f(x)
    return 2 * x * t.exp(x) + f(x)

In [16]:
x = t.randn(3, 4, requires_grad=True)
y = f(x)
y

tensor([[5.1295e-01, 1.8462e-02, 5.2898e-01, 3.6454e+00],
        [5.3742e-01, 7.7192e-02, 5.0172e-01, 2.3497e+00],
        [1.4202e-02, 1.4914e-05, 1.7136e+00, 3.6328e-03]],
       grad_fn=<MulBackward0>)

In [17]:
y.backward(t.ones(y.size()))  # 这个参数有点类似于上一级得到的梯度
x.grad

tensor([[-1.4007e-01,  3.0810e-01, -8.9271e-02,  1.0267e+01],
        [-4.8579e-02,  7.0550e-01, -1.6811e-01,  7.2850e+00],
        [-2.0948e-01,  7.7537e-03,  5.7264e+00,  1.2776e-01]])

In [18]:
# autograd的计算结果与利用公式的手动计算结果一致
grad_f(x)

tensor([[-1.4007e-01,  3.0810e-01, -8.9271e-02,  1.0267e+01],
        [-4.8579e-02,  7.0550e-01, -1.6811e-01,  7.2850e+00],
        [-2.0948e-01,  7.7537e-03,  5.7264e+00,  1.2776e-01]],
       grad_fn=<AddBackward0>)

### 3.2.2 计算图

PyTorch中`autograd`的底层采用了计算图，计算图是一种特殊的有向无环图（DAG），用于记录算子与变量之间的关系。一般用矩形表示算子，椭圆形表示变量。如表达式$ \textbf {z = wx + b}$可分解为$\textbf{y = wx}$和$\textbf{z = y + b}$，其计算图如图3-3所示，图中`MUL`，`ADD`都是算子，$\textbf{w}$，$\textbf{x}$，$\textbf{b}$即变量。

![图3-3:computation graph](imgs/com_graph.svg)

如上有向无环图中，$\textbf{X}$和$\textbf{b}$是叶子节点（leaf node），这些节点通常由用户自己创建，不依赖于其他变量。$\textbf{z}$称为根节点，是计算图的最终目标。利用链式法则很容易求得各个叶子节点的梯度。
$${\partial z \over \partial b} = 1,\space {\partial z \over \partial y} = 1\\
{\partial y \over \partial w }= x,{\partial y \over \partial x}= w\\
{\partial z \over \partial x}= {\partial z \over \partial y} {\partial y \over \partial x}=1 * w\\
{\partial z \over \partial w}= {\partial z \over \partial y} {\partial y \over \partial w}=1 * x\\
$$
而有了计算图，上述链式求导即可利用计算图的反向传播自动完成，其过程如图3-4所示。

![图3-4：计算图的反向传播](imgs/com_graph_backward.svg)


在PyTorch实现中，autograd会随着用户的操作，记录生成当前variable的所有操作，并由此建立一个有向无环图。用户每进行一个操作，相应的计算图就会发生改变。更底层的实现中，图中记录了操作`Function`，每一个变量在图中的位置可通过其`grad_fn`属性在图中的位置推测得到。在反向传播过程中，autograd沿着这个图从当前变量（根节点$\textbf{z}$）溯源，可以利用链式求导法则计算所有叶子节点的梯度。每一个前向传播操作的函数都有与之对应的反向传播函数用来计算输入的各个variable的梯度，这些函数的函数名通常以`Backward`结尾。下面结合代码学习autograd的实现细节。

In [19]:
x = t.ones(1)
b = t.rand(1, requires_grad=True)
w = t.rand(1, requires_grad=True)
y = w * x  # equal to y = w.mul(x)
z = y + b  # equal to z = y.add(b)

In [20]:
x.requires_grad, b.requires_grad, w.requires_grad

(False, True, True)

In [21]:
y.requires_grad

True

In [23]:
x.is_leaf, w.is_leaf, b.is_leaf

(True, True, True)

In [24]:
y.is_leaf, z.is_leaf

(False, False)

In [25]:
z.grad_fn

In [26]:
# next_functions保存grad_fn的输入，是一个tuple，tuple的元素也是Function
# 第一个是y，它是乘法(mul)的输出，所以对应的反向传播函数y.grad_fn是MulBackward
# 第二个是b，它是叶子节点，由用户创建，grad_fn为None，但是有
z.grad_fn.next_functions 

((<MulBackward0 at 0x1b0fc291eb0>, 0), (<AccumulateGrad at 0x1b0fc291fd0>, 0))

In [27]:
z.grad_fn.next_functions[0][0] == y.grad_fn

True

In [28]:
# 第一个是w，叶子节点，需要求导，梯度是累加的
# 第二个是x，叶子节点，不需要求导，所以为None
y.grad_fn.next_functions

((<AccumulateGrad at 0x1b0ffef87f0>, 0), (None, 0))

In [29]:
# 叶子借点的grad_fn是None
w.grad_fn, x.grad_fn

(None, None)

计算w的梯度的时候，需要用到x的数值(${\partial y\over \partial w} = x $)，这些数值在前向过程中会保存成buffer，在计算完梯度之后会自动清空。为了能够多次反向传播需要指定`retain_graph`来保留这些buffer。

In [30]:
# 使用retain_graph来保存buffer
z.backward(retain_graph=True)
w.grad

tensor([1.])

In [31]:
# 多次反向传播，梯度累加，这也就是w中AccumulateGrad标识的含义
z.backward()
w.grad

tensor([2.])

PyTorch使用的是动态图，它的计算图在每次前向传播时都是从头开始构建，所以它能够使用Python控制语句（如for、if等）根据需求创建计算图。这点在自然语言处理领域中很有用，它意味着你不需要事先构建所有可能用到的图的路径，图在运行时才构建。

In [32]:
def abs(x):
    if x.data[0] > 0: return x
    else: return -x

x = t.ones(1, requires_grad=True)
y = abs(x)
y.backward()
x.grad

tensor([1.])

In [33]:
x = -1 * t.ones(1)
x = x.requires_grad_()
y = abs(x)
y.backward()
print(x.grad)

tensor([-1.])


In [34]:
y

tensor([1.], grad_fn=<NegBackward>)

In [35]:
x

tensor([-1.], requires_grad=True)

In [36]:
x.requires_grad

True

In [37]:
def f(x):
    result = 1
    for ii in x:
        if ii.item() > 0:
            result = ii * result
    return result

x = t.arange(-2, 4, dtype=t.float32).requires_grad_()
y = f(x)  # y = x[3] * x[4] * x[5]
y.backward()
x.grad

tensor([0., 0., 0., 6., 3., 2.])

变量的`requires_grad`属性默认为False，如果某一个节点requires_grad被设置为True，那么所有依赖它的节点`requires_grad`都是True。这其实很好理解，对于$ \textbf{x}\to \textbf{y} \to \textbf{z}$，x.requires_grad = True，当需要计算$\partial z \over \partial x$时，根据链式法则，$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial y} \frac{\partial y}{\partial x}$，自然也需要求$ \frac{\partial z}{\partial y}$，所以y.requires_grad会被自动标为True. 



有些时候我们可能不希望autograd对tensor求导。认为求导需要缓存许多中间结构，增加额外的内存/显存开销，那么我们可以关闭自动求导。对于不需要反向传播的情景（如inference，即测试推理时），关闭自动求导可实现一定程度的速度提升，并节省约一半显存，因其不需要分配空间计算梯度。

In [38]:
x = t.ones(1, requires_grad=True)
w = t.rand(1, requires_grad=True)
y = x * w
# y依赖于w，而w.requires_grad = True
x.requires_grad, w.requires_grad, y.requires_grad

(True, True, True)

In [39]:
t.no_grad??

Init signature: t.no_grad()
Source:        
class no_grad(_DecoratorContextManager):
    r"""Context-manager that disabled gradient calculation.

    Disabling gradient calculation is useful for inference, when you are sure
    that you will not call :meth:`Tensor.backward()`. It will reduce memory
    consumption for computations that would otherwise have `requires_grad=True`.

    In this mode, the result of every computation will have
    `requires_grad=False`, even when the inputs have `requires_grad=True`.

    This context manager is thread local; it will not affect computation
    in other threads.

    Also functions as a decorator. (Make sure to instantiate with parenthesis.)


    Example::

        >>> x = torch.tensor([1], requires_grad=True)
        >>> with torch.no_grad():
        ...   y = x * 2
        >>> y.requires_grad
        False
        >>> @torch.no_grad()
        ... def doubler(x):
        ...     return x * 2
        >>> z = doubler(x)
        >>> z.requires

In [40]:
t.set_grad_enabled(False)
x = t.ones(1)
w = t.rand(1, requires_grad=True)
y = x * w
# y依赖于w和x, 虽然w.requires_grad=True,但是y的requires_grad为False
x.requires_grad, w.requires_grad, y.requires_grad

(False, True, False)

In [41]:
# 恢复默认设置
t.set_grad_enabled(True)

如果我们想要修改tensor的数值，但是又不希望被autograd记录，那么我么可以对tensor.data进行操作